# Most Frequently Used next-app prediction

In [15]:
from torch import optim
from data.dataloader import DatasetLoader, PretrainLoader
from torch.utils.data import DataLoader
import torch
from models.BiGRU import BiGRU
from models.PreTrainEmbedding import PreTrainEmbedding
from torch import nn
from tqdm import tqdm
import pandas as pd
import numpy as np

## 1. Dataset uitlezen

In [5]:
removeapps = ["Screen on (locked)",
              "Screen off (locked)",
              "Screen on (unlocked)",
              "Screen off",
              "Samsung Experience Service",
              "Package installer",
              "System UI",
              "Customisation Service",
              "Configuration update",
              "EmergencyManagerService",
              "DeviceKeystring",
              "Samsung Keyboard",
              "HwModuleTest",
              "Device shutdown",
              "Device boot"]
dataset = DatasetLoader("eventlog/phone_usage_cleaned.csv",
                        seq_length=1)
dataset.clean(removeapps=removeapps)
train_dataloader = DataLoader(dataset)

## pre-train de MFU waarden voor 2000 eerste applicaties

In [39]:
appcount = np.zeros(shape=(200,), dtype=np.int) # index==app_index, value=keer_geopend
nr_opened_apps = 0

for prev_app_index, _ in train_dataloader:

    appcount[prev_app_index.item()] += 1

    nr_opened_apps+=1
    if nr_opened_apps==2000:
        break

print(appcount)
print("nultoken is voor padding")

[  0  86 146   1 184   5  14  26 452   1   6 369  35   4  17  66  43   2
  36   6  44 122   1   9   9  20   1   8   4   2  41   3  28   4   4  12
   1  35  15   2   2   2   2  16   1   6   4   2   1   2   3   1  10   1
  20  18  30   3   2   1   5   4   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
nultoken is voor padding


## Maak predicties en update MFU waarden voor de rest van de applicaties

In [41]:
nr_opened_apps = 0
running_accuracy_list = []

for _, next_app_index in train_dataloader:

    nr_opened_apps+=1
    if nr_opened_apps > 2000:
        mostfreq = appcount.argsort()[-5:][::-1]
        running_accuracy_list.append(next_app_index.item() in mostfreq)

        appcount[next_app_index.item()] += 1

## Accuracy uitschrijven naar comparison

In [71]:
AccuracyComparison = pd.read_csv("outputdata/accuracycomparison.csv", index_col=False)
AccuracyComparison["MFU"] = running_accuracy_list
AccuracyComparison.to_csv("outputdata/accuracycomparison.csv", index=False)
AccuracyComparison.head()


,MFU
0,True
1,False
2,False
3,False
4,False
